# Light-Matter interaction computational exercise 

Written by Milaim Kas <br>
If you have any questions, you can send an email to milaim.kas@desy.de

## TO DO:
- Add molecular orbital picture ?

### Module III: Chemical sensitivity in carbon K edge structure 

In this third module, you will use a quantum chemistry program to: 

- calculate electron binding energies of core **electron**
- calculate excited states properties using Hartree-Fock orbitals
- plot an X-ray emission/absorption spectrum
- investigate the chemical sensitivity of the carbon K-edge spectra

The quantum chemistry module that is used is PySCF (\url{https://sunqm.github.io/pyscf/}). You will not directly deal with PySCF's syntax but rather use a class that will allow you to perform the needed calculations and extract the results through a simplified syntax. An example with the CO molecule is provided. 

The questions have been highlighted in green, red and blue, with the following meaning:
- green: the answer does not have to appear in the report.
- blue: these question involve plotting. They have to appear as figure in the report, along with a caption, legend and axis title. In addition, a short discription of the observed features must be given.
- red: an written answer must be given.   

### Sub-routine

The relevant class and functions are given below, there is no need to go through it but an interested reader will find extended comments to understand the content.

In [ ]:
import pyscf
import numpy as np
from pyscf import scf,gto
import matplotlib.pyplot as plt


In [ ]:
# Molecule object
class Molecule:
    def __init__(self, name_molecule):
    
       '''
        initialize molecule and perform Hartree Fock and MOM (maximal overlap method) calculation with the Gaussian 6-31g basis set
        arg:
         - name_molecule = 'CO', 'HNO', 'CNH5', 'CH4', 'CO2', 'CH2O', 'C2H6', 'C2H4', 'C2H2'
       '''
    
       # Define molecular geometry for several compounds
       self.mol       = pyscf.gto.Mole() 
       if name_molecule == 'CO':
         self.mol.atom  = '''
         C 0. 0. 0.
         O 0. 0. 1.147 
         ''' 
       elif name_molecule == 'HNO':
         self.mol.atom = '''
         H  -0.938530   0.9102970  0.0000000
         N  0.0625690   0.5847110  0.0000000
         O  0.0625690  -0.6254090  0.0000000 
         '''
       elif name_molecule == 'CO2':
         self.mol.atom = '''
         C  0.  0.  0.
         O  0.  0.  -1.16
         O  0.  0.  1.16
         '''
       elif name_molecule == 'CH2O':
         self.mol.atom = '''
         O  0.0000000  0.0000000    0.6840550
         C  0.0000000  0.0000000   -0.5377120
         H  0.0000000  0.9482250   -1.1230830
         H  0.0000000  -0.9482250  -1.1230830
         '''
       elif name_molecule == 'C2H2':
         self.mol.atom = '''
         C	0.0000000	0.0000000	0.6146230
         C	0.0000000	0.0000000	-0.6146230
         H	0.0000000	0.0000000	1.6926020
         H	0.0000000	0.0000000	-1.6926020
         '''
       elif name_molecule == 'C2H6':
         self.mol.atom = '''
         C	0.0000000	0.0000000	0.7696050
         C	0.0000000	0.0000000	-0.7696050
         H	0.0000000	1.0302260	1.1677980
         H	-0.892202	-0.515113	1.1677980
         H	0.8922020	-0.515113	1.1677980
         H	0.0000000	-1.030226	-1.1677980
         H	-0.892202	0.5151130	-1.1677980
         H	0.8922020	0.5151130	-1.1677980
         '''
       elif name_molecule == 'C2H4':
         self.mol.atom = '''
         C	0.0000000	0.0000000	0.6770110
         C	0.0000000	0.0000000	-0.6770110
         H	0.0000000	0.9347540	1.2488340
         H	0.0000000	-0.934754	1.2488340
         H	0.0000000	-0.934754	-1.2488340
         H	0.0000000	0.9347540	-1.2488340
         '''
       elif name_molecule == 'CNH5':   
         self.mol.atom = '''
         C	0.05079000	0.7054590	0.0000000
         N	0.05079000	-0.758502	0.0000000
         H	-0.9425740	1.1646480	0.0000000
         H	0.58943700	1.0605680	0.8772540
         H	0.58943700	1.0605680	-0.8772540
         H	-0.4482880	-1.104511	-0.8089300
         H	-0.4482880	-1.104511	0.8089300
         '''   
       else:
         print('Molecule not found in library')
       
       # define gaussian basis set
       self.mol.basis = '''6-31g'''
       # use symmetry     
       self.mol.symmetry = True
       # no output
       self.mol.verbose = 0
       # build one and two electron integrals
       self.mol.build()
       
       # do Restricted Hartree Fock calculation
       # results are stored in the object mf
       self.mf = pyscf.scf.RHF(self.mol)
       self.mf.kernel()
       # store occupation numbers
       self.occ = self.mf.mo_occ

    
    def MO_energy(self):
       '''
       print Hartree Fock MO (molecular orbital) energies
       '''
       return self.mf.mo_energy

    def print_MO(self):
       '''
       print information on the Hartree Fock MO
       '''
       self.mf.analyze(verbose=5)
    
    def tdm(self,i,a):
        '''
        calculates 
        - the tdm between two MOs "i" and "a"
        where "i" is occupied and "a" virtual MO
        - the oscillator strength relate to that transition
        '''

        # tdm in x,y and z direction for all given MO transition
        tdm = np.zeros((3))
        osci = 0

        # dipole integrals in AOs basis set for the 3 cartesians components
        with self.mol.with_common_orig((0, 0, 0)):
            ao_dip  = self.mol.intor_symmetric('int1e_r', comp=3)
        
        
        tdm[0] = np.einsum('j,k,jk', self.mf.mo_coeff[:, i], self.mf.mo_coeff[:, a].conj(), ao_dip[0])
        tdm[1] = np.einsum('j,k,jk', self.mf.mo_coeff[:, i], self.mf.mo_coeff[:, a].conj(), ao_dip[1])
        tdm[2] = np.einsum('j,k,jk', self.mf.mo_coeff[:, i], self.mf.mo_coeff[:, a].conj(), ao_dip[2])

        osci += (2/3)*abs(self.mf.mo_energy[a]-self.mf.mo_energy[i])
        osci *= np.sum(abs(tdm)**2)

        return tdm, osci   

    def MOM(self,i,a):
          '''
          Maximal overlap calculation for a i --> a excited determinant
          '''
          # define excited singlet occupation for alpha spin
          occ = np.zeros((2, self.occ.size))
          occ[:, self.occ==2] = 1
          occ[1][a] = 1 # virtual orbital a
          occ[1][i] = 0 # occupied orbital i
          occ_es = occ[0][:] + occ[1][:]
          # store mo coefficients in ROHF format
          mo_coeff = self.mf.mo_coeff
          # new SCF calculation
          es_hf = scf.ROHF(self.mol)
          # construct new density matrix with excited occ vector
          dm_es = es_hf.make_rdm1(mo_coeff, occ_es)
          # Apply mom occupation principle
          es_hf = scf.addons.mom_occ(es_hf, mo_coeff, occ)
          # Start new SCF with new density matrix
          es_hf.scf(dm_es)
          
          # excitation energy
          Ex_ene = es_hf.e_tot - self.mf.e_tot  

          # new MOs energy
          #MO_ene  = es_hf.mo_energy

          return Ex_ene



### Theory


#### **Mean field approach for the ground state**

Different approaches are used in order to explicitly treat multi-electron systems. Most of them are based on expansion over single particle functions. These single particle functions are eigenfunctions of some one-electron operator where the explicit coupling (or correlation) between the particles (here electrons) is neglected. The most common approach is the Hartree-Fock method were the interaction between the electrons is approximated as a mean field. An Hartree-Fock calculation produces a set of single-particle function $\phi_i$ (also called molecular orbitals: MOs) that are expressed as linear combination of basis functions $\chi_{j}$ which are referred to as atomic orbitals (AOs):

$$ \phi_i = \sum^{N_{basis}}_{j} c_{ij}\chi_{j}$$

$N_{basis}$ is the size of the AO basis and $c_{ij}$ are the optimized coefficients, usually called molecular orbital coefficients.
The MOs are separated into occupied orbitals, characterized by negative energy, and unoccupied orbitals (positive energy). They are eigenfunctions of the Fock operator (see Chapter $8.7-8.8$ of the lecture notes) <br><br>

#### **Treatment of excited states**

An accurate treatment of excited states involves expansion of the wave function unto basis of deterinant (cf. CI method: Chapter $8.2$ in the lecture notes) or time-dependent approaches. Here we will deal with excited states using a much simpler (and faster) method. 
In the Hartree-Fock (HF) method, an approximation of excitation energy can be obtain by taking the sum of the absolute value of the two MOs energy involved in the transition. This is usually referred to as Koopman's theorem. 

For example the first valence excited states of a molecule is usually characterize by a single excitation from the HOMO (highest occupied molecular orbital) to the LUMO (lowest unoccupied molecular orbital).
In the CO molecule, within a define basis set, the LUMO has an energy of $0.154$ au and the HOMO has an energy of $-0.545$ au. The excitation energy associated to this HOMO --> LUMO transition is therefore $abs(-0.545)+abs(0.154)=0.699$ au

One should be aware that this crude approximation neglects many effects: electron correlation and orbital relaxation among others. Even a quantitative excitation energy is rarely obtained. For example, the relaxation correction to this model can be quite big with around 15eV for C, 1 eV for N and 0.2 eV for Oxygen (Stohr J., NEXAFS Spectroscopy, Springer, 1996, pp 22). However, considering the demonstrative purposes of this exercise, the HF model will be enough. <br><br>

#### **K-edge spectrum**

K-edge spectrum refers to a spectrum obtained when a X-Ray light excites a $1s$ inner core orbital. The resulting excited state may lay below the ionization potential (IP) of the $1s$ orbital, leading to pre-edge and edge spectral lines, or above the IP, leading to above-edge structure. In the one-electron picture, these spectral features arise due to transitions from the 1s core orbital to valence excited MOs.

In the present exercise you will calculate the excitation energies and transition dipole moment of core-to-valence transition for various molecules. The idea is to understand which properties, unique to each molecule, can be retrieved by means of X-ray spectroscopy. <br><br>

#### **Spectral line intensity**

The height of a spectral line is proportional to the transition moment between the two states involved in the transition. Different quantities are used to characterize the intensity of a transition. The oscillator strength can be define as:

$$f_{ik}=\frac{2}{3}\frac{\Delta E}{g_i} \sum_{\alpha}|<\Psi_k|\hat{d}_{\alpha}|\Psi_i>|^2$$

where $\Delta E$ is the excitation energy from the initial state $i$ to the final state $k$, $g_i$ is the degeneracy of the initial state and $\hat{d}_{\alpha}$ is $\alpha$ component of the dipole operator. <br><br>

#### **Chemical bond**

The chemical bond is the force that held atoms together to form molecules. The strength of a chemical bond $A-B$ can be define by the energy needed to seperate both fragment $A$ and $B$. In many case, the MOs involving valence electrons, hold information on the bonding character (strong, weak , etc) of the molecule.

As an example, the carbon atom can bond to an other carbon via different types of chemical bond: single, double and triple bond. Information of the type of chemical bond can be obtained from the valence MOs (the HOMO and LUMO).



### **Calculating a K-edge spectra: example with the CO molecule**


#### **Step 1: Hartree Fock calculation and analysis of the MOs**

First we create an python object that we call "CO". This object will hold information on the Hartree-Fock results of the CO molecule.

In [ ]:
CO = Molecule('CO')

The information of the single particle functions: their energy and composition in terms of atomic orbitals can be obtained from the object 'CO'.

Printed information:
- Total energy for the HF ground state (in au).
- After the header "$****$ MO energy $****$" the occupation and energy of the one-electron function $\phi_i$ (MO #) are shown. For example the first one-electron function is labeled MO #1, it has an energy of around $-20.58$ au and is occupied by two electrons. The point symmetry of the MO is also given, for diatomic molecules they correspond to the usual $\sigma$, $\pi$ notation with: $A1 = \sigma$, $E1x=\pi_x$ and $E1y=\pi_y$. 
- After the header "$**$ MO coefficients (expansion on meta-Lowdin AOs) $**$", the expansion coefficients $c_{ij}$ are given in columns, one for each MO ($\phi_i$). Each row refers to a AOs ($\chi_j$). For example one can see that the MO #1 is mainly composed by the $1s$ AO of oxygen, and the MO #2 is the $1s$ AO of carbon. 
- The  $**$ Mulliken pop on meta-lowdin orthogonal AOs $**$ section gives information on the electronic populations of the single atomic orbitals and on the charge distribution within the molecule.

In [ ]:
CO.print_MO()

**** SCF Summaries ****
Total Energy =                        -112.666599766068515
Nuclear Repulsion Energy =              22.145166629607672
One-electron Energy =                 -197.115458973601591
Two-electron Energy =                   62.303692577925403
TODO: total wave-function symmetry for Coov
occupancy for each irrep:     A1  E1x  E1y
                               5    1    1
**** MO energy ****
MO #1 (A1 #1), energy= -20.6863637669333 occ= 2
MO #2 (A1 #2), energy= -11.4029546234066 occ= 2
MO #3 (A1 #3), energy= -1.54740479404474 occ= 2
MO #4 (A1 #4), energy= -0.795961432315603 occ= 2
MO #5 (E1x #1), energy= -0.634418637982631 occ= 2
MO #6 (E1y #1), energy= -0.634418637982631 occ= 2
MO #7 (A1 #5), energy= -0.552519146745798 occ= 2
MO #8 (E1x #2), energy= 0.135235290806966 occ= 0
MO #9 (E1y #2), energy= 0.135235290806966 occ= 0
MO #10 (A1 #6), energy= 0.402938743400825 occ= 0
MO #11 (E1x #3), energy= 0.727847811784471 occ= 0
MO #12 (E1y #3), energy= 0.727847811784467 occ= 0
M

The MOs energy (given in au) can be direclty access and store into an array using the following function


In [ ]:
print(CO.MO_energy())

[-20.57876197 -11.3239252   -1.53415502  -0.7837049   -0.62680263
  -0.62680263  -0.54480585   0.15376451   0.15376451   0.48892028
   0.92622182   0.92622182   1.08193724   1.32132381   1.79403376
   1.79789452   1.79789452   3.29890443]


You can see that there are 7 and 11 occupied and unoccupied MOs, respectively

#### **Step 2: getting the excitation energies**

Let us now calculate the oxygen core-to-valence excitation energies in CO.

The first MOs corresponds to the $1s$ AO of O. Within our basis set, the LUMO has an energy of $0.1537$ au and the $1s_O$ MOs has an energy of $-20.58$ au (see previous point). The excitation energy associated to this $1s_O$ --> LUMO transition is: 

In [ ]:
DE_CO_O = np.zeros((5))
DE_CO_O[0] = abs((CO.MO_energy()[7]-CO.MO_energy()[0])) 
print(DE_CO_O[0])

20.8215990577403


Higher core excitation energies are obtained by "promoting" the $1s_O$ electron to higher virtual orbitals. The obtained serie of excitation energies corresponds to the K edge of oxygen in CO.  

In [ ]:
for i in range(5):
  DE_CO_O[i]=abs(CO.MO_energy()[7+i]-CO.MO_energy()[0])

print(DE_CO_O) # in au
print(DE_CO_O*27.2114) # in eV

[20.82159906 20.82159906 21.08930251 21.41421158 21.41421158]
[566.5848606  566.5848606  573.86944633 582.71067695 582.71067695]


Note that whithin this approximation, the calculated excitation energies are larger than the Koopman's ionization potential (IP) of the $1s_O$ orbital ($20.58$ au). 

In order to get better excitation energies, one can use the maximal overlap method (MOM, see *T. B. Gilbert et al, J. Phys. Chem. A 2008, 112, 50, 13164*) that account for orbital relaxation. 

In [ ]:
DE_CO_O_MOM = np.zeros((5))
for i in range(5):
  DE_CO_O_MOM[i]=CO.MOM(0,7+i)

print(DE_CO_O_MOM)
print(DE_CO_O_MOM*27.2114)

[19.67754425 19.67754425 20.05471035 20.34553497 20.34553497]
[535.45352761 535.45352761 545.71674508 553.63049019 553.63049019]


As you can see, the difference in excitation energy is around 30 eV ! In addition or improved excitation energies are now below the IP of the $1s_O$ orbital.

However, we will stick to our rough HF approximation for the remaining exercises. 

<font color='green'> **1) Calculate the carbon K-edge in CO (calculate the 5 first core-to-valence excitation energies)**

hint: follow the same procedure as above but use the $1s$ orbital of carbon instead of oxygen

#### **Step 3: getting the transition dipole moments and oscillator strength**

Transition dipole moment and oscillator strength can be extracted for a given pair of occupied/virtual MO. This is done using the $tdm$ function: $CO.tdm(i,a)$ which returns the three cartesian components of the transition dipole moment and the oscillator strength for a $i$ --> $a$ transition. 

For the $1s_O$ --> LUMO transition:



In [ ]:
print(CO.tdm(0,7)[0]) # transition dipole moment
print()
print(CO.tdm(0,7)[1]) # oscillator strength

[-0.03108124  0.          0.        ]

0.013409713214897425


<font color='green'> **2) Calculate the transition dipole moments and oscillator strength for the transistion calculated in 1).**


<font color='blue'>**3a) Plot the X-Ray absorption spectrum at the C K-edge of CO using the excitation energies and oscillator strength obtained in 1) and 2).**

hint i : plot vertical lines using the $plt.vlines()$ function. <br>
hint ii: the position of the line is the excitation energy while the height can be given by the oscillator strength. <br>

<font color='blue'>**3b) Plot the X-Ray absorption spectrum at the O K-edge of CO.**

<font color='brown'>**3c) In points 3a and 3b we have plotted the oscillator strengths relative to 5 excitations. Why do we see only three lines in the plotted spectra? In both the O and C K-edge the final orbitals are the same. Comparing the two spectra you can see that, the oscillator strenghts of the transitions with the same final MO is however very different both in absolute value and relative intensity distribution. Why is it so?**

hint: think about the composition of the MOs in terms of AOs.

#### **Step 4: line broadening**

In a real spectrum, the spectral lines always have a finite, non-zero width. This is reffered to as line broadening and arises from different effects, such as : 

- Natural broadening (related to the time-energy uncertainity principle) 
- Collision broadening
- Doppler effect

the two first effects are related to the finite life-time of the excited states. The first case arises from the intrinsic finite life-time (coupling with the field or other states) and the second is due to collision-induced decay (in a gas for example). One can define a phenomenological decay probability $\Gamma$, which is related to the life-time $\tau$ by $\Gamma=1/\tau$, that account for all possible decay channel. In the following, $\Gamma$ is considered to be the FWHM (full width at half maximum) of the line. 

The line shape related to the finite life-time is given by a Lorentzian function:

$$ g(E) = \frac{(\Gamma/2)^2}{(E_0-E)^2+(\Gamma/2)^{2}} $$

where $E_0$ is the excitation energy. A typical life-time for a bound valence electronic transition is $10^{-8}s$ while core excited states have shorter life-time, in the order of $10^{-15}s$.

$1fs=10^{-15}s=41.34$ au



<font color='blue'>**4) Use the last given expression to simulate a more realistic C K-edge spectrum for CO by adding the line broadening to the vertical lines.**

hint i: use the results you got in 3a) and 3b) <br>
hint ii: one is usualy not interested in absolute value for the line intensities, only in the ratio. Normalize your lines with respect to the most intense transition. 

### **Carbon K-edge spectra for various molecules containing carbon atoms**

<font color='brown'>**5) Extract the binding energy of the $1s_C$ orbital for the C2H2, C2H4 and C2H6 molecules. How do they compare ? How would you explain it ?**

hint: follow the procedure given above for CO
- step 1: create a Molecule object and analyze the MOs, identify the $1s_C$ and the unoccupied MOs

<font color='blue'>**6) Calculate and plot a C K-edge spectrum for the C2H2, C2H4 and C2H6 molecules**

hint i: follow the steps given in the CO example
- step 2: store the core-to-valence excitation energies for 5 transitions
- step 3: store the oscillator strength for the 5 transitions
- step 4: add the line broadening 

hint ii: when plotting the final spectrum, you will have to add the spectra of each molecule.

hint iii: do not forget to normalize you final spectrum (with respect to the most intense line).

Step 2:

Step 3:

Step 4:

<font color='brown'>**7) Could you use these spectral lines to distinguish the different compounds? Considering your observation in point 5), how do you explain the difference in the position and intensity of the spectral lines.  What type of conclusions can you make on the final MOs?**

<font color='brown'>**8) Based on your answer in the previous question, what type of information on the molecule can you retrieve from the spectral analysis?**

<font color='brown'>**9) The spectral lines would also be different in a UV/visible spectra (which involves HOMO --> LUMO transitions). Explain the advantage of using X-ray absorbtion.** 

hint: X-ray involves a core to valence transition 

<font color='brown'>**10) What happens if the life-time of the excited states decreases ? In an experiment, what could cause such increase ?**

<font color='green'>**Bonus: investigate the influence of orbital relaxation using the MOM method on the spectrum obtained in 6)**